HW5 Мельчук А.Б.
Обучить детектор объектов с помощью TensorFlow Object Detection API
Библиотеки: [Python, Tensorflow]

In [0]:
%tensorflow_version 1.x
import tensorflow as tf

In [0]:
!git clone https://github.com/Dju999/TFFashionDetection.git
!pip install lxml
!pip install -U -q PyDrive
!pip install tqdm

#### Качаем датасет [DeepFashion](http://mmlab.ie.cuhk.edu.hk/projects/DeepFashion.html) и монтируем его

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from TFFashionDetection.utils.colab_fs import GoogleColabFS

fs = GoogleColabFS()

In [0]:
!rm -rv ./fashion_data
!cp -rv "drive/My Drive/Category and Attribute Prediction Benchmark/" ./TEMP
!mv ./TEMP ./fashion_data


In [0]:
!unzip ./fashion_data/Img/img.zip -d ./fashion_data/Img

#### Tensorflow Object Detection API

In [0]:
!pip install tf_slim

In [0]:
# загружаем репозиторий, добавляем в PYTHONPATH
! cd /content; git clone https://github.com/tensorflow/models.git
# установка зависимостей для object detection тут 
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!cd /content; git clone https://github.com/cocodataset/cocoapi.git; cd cocoapi/PythonAPI; make; cp -r pycocotools /content/models/research/
!cd /content/models/research; protoc object_detection/protos/*.proto --python_out=.
# проверка
!cd /content/models/research; export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python object_detection/builders/model_builder_test.py

#### Создание структуры данных

In [0]:
import sys
import os

import numpy as np

API_PATH = os.path.join('/content', 'models/research')
sys.path.append(API_PATH)

DETECTOR_PATH = os.path.join('/content', 'TFFashionDetection')
sys.path.append(DETECTOR_PATH)

from TFFashionDetection.data_preparator import DataPreparator
from TFFashionDetection.utils.ssd_config import write_config

data_preparator = DataPreparator()
data_preparator.build()
write_config('ssd_mobilenet_v2_coco_2018_03_29')

In [0]:
# скачиваем модель (предобученную)
!python /content/TFFashionDetection/utils/download_tf_zoo_model.py --name ssd_mobilenet_v2_coco_2018_03_29 --dir /content

### Обучаем модель детекции

In [0]:
!export PYTHONPATH=$PYTHONPATH:/content/models/research/slim:/content/models/research/;python /content/models/research/object_detection/legacy/train.py --logtostderr --pipeline_config_path=/content/data_dir/tf_api.config --train_dir=/content/data_dir/checkpoints

### Сохраняем чекпоинт на Гугл-диск

In [0]:
!cd /content/data_dir; zip -r checkpoint_save_20180514.zip checkpoints/*
import os
fs = GoogleColabFS()

file_name = os.path.join('/content/data_dir', 'checkpoint_save_20180514.zip')

fs.load_to_drive(file_name)

### Скачиваем с Гугл-диска и экспортируем модель

In [0]:
import logging
import os

logger = logging.getLogger(__name__)

fs.load_file_from_drive('/content', 'checkpoint_save_20180514.zip')
fs.unzip_file('/content', 'checkpoint_save_20180514.zip')

!mkdir /content/deep_detection_model

# экспортируем модель
!export PYTHONPATH=$PYTHONPATH:/content/models/research/slim:/content/models/research/;python /content/models/research/object_detection/export_inference_graph.py --input_type image_tensor --pipeline_config_path=/content/data_dir/tf_api.config --trained_checkpoint_prefix=/content/checkpoints/model.ckpt-2108 --output_directory inference_graph

In [0]:
from IPython.display import Image
import os

file_path = os.path.join('/content/data_dir/images', os.listdir('/content/data_dir/images')[6])

Image(file_path)

### Пример детекции

In [0]:


from TFFashionDetection.tf_object_detector import ObjectDetector
import numpy as np

oject_detector = ObjectDetector('/content/frozen_model/ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb', '/content/data_dir/annotations/label_map.pbtxt')

oject_detector.object_detection(image_dir='/content/data_dir/images', filename=file_path)
[[i['category_box'],i['img_name'],i['category_id'], i['category_proba']] for i in oject_detector.img_detections[:5]]
#oject_detector.img_detections
np.unique([i['category_name'] for i in oject_detector.img_detections])

In [0]:
import sys
import os

import matplotlib.pyplot as plt
plt.switch_backend('agg')

sys.path.append(os.path.join('/content', 'models/research'))
from object_detection.utils import visualization_utils as vis_util

from PIL import Image as Pil_image

%matplotlib inline
boxes = np.array([oject_detector.img_detections[3]['category_box']])

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)
# загружаем картинку и превращаем в массив
image = Pil_image.open(file_path)
image_np = load_image_into_numpy_array(image)
# накладываем на массив bounding boxes
vis_util.draw_bounding_boxes_on_image_array(image_np, boxes)
# сохраняем картинку на диск
result_file_path = os.path.join('/content', 'test.png')
vis_util.save_image_array_as_png(image_np, result_file_path)
# виуализируем картинку, которую сохранили
from IPython.display import Image
Image(result_file_path)

In [0]:
ls /content

### Получаем эмбеддинги изображений

In [0]:
from TFFashionDetection import feature_extraction as ftrs
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

ftrs.get_dir_embeddings(model_dir='/content/deep_detection_model', data_dir='/content/data_dir/images', num_top_predictions=1, max_images=500)

In [0]:
import pickle

embeds_file = '/content/data_dir/img_embeddings.pkl'

embeds = pickle.load(open(embeds_file, 'rb'))

In [0]:
from sklearn.neighbors import KDTree
import numpy as np
num_embeds = len(embeds)
embed_index = dict(zip(embeds.keys(), range(num_embeds)))
inverted_index = {j:i for i,j in embed_index.items()}

embed_matrix = np.vstack([embeds[k] for k in embed_index])

kdt = KDTree(embed_matrix, leaf_size=30, metric='euclidean')

In [0]:
test_id = embed_index[list(embed_index.keys())[620]]

test_example = embed_matrix[test_id]
sim_query = kdt.query(test_example.reshape(1,-1), k=5, return_distance=False)

In [0]:
from IPython.display import Image
import os

#file_path = '/content/data_dir/images/americana_skinny_jeans_img_00000030.jpg'
#file_path = '/content/data_dir/images/zippered_sweatpants_img_00000046.jpg'
file_path = inverted_index[sim_query[0][0]]

Image(file_path)

In [0]:
file_path = inverted_index[sim_query[0][1]]

Image(file_path)